Now we parse the content of the JSON file with the QASM circuits

In [1]:
using JSON
using ITensors



In [2]:

function h_gate(sites, i::Int)
    os = OpSum()
    os += "H", i
    return MPO(os, sites)
end

function rz_gate(sites, i::Int, θ::Float64)
    os = OpSum()
    os += cos(θ / 2.0), "I", i
    os += -im * sin(θ / 2.0), "Z", i
    return MPO(os, sites)
end

function rx_gate(sites, i::Int, θ::Float64)
    os = OpSum()
    os += cos(θ / 2.0), "I", i
    os += -im * sin(θ / 2.0), "X", i
    return MPO(os, sites)
end

function ry_gate(sites, i::Int, θ::Float64)
    os = OpSum()
    os += cos(θ / 2.0), "I", i
    os += -im * sin(θ / 2.0), "Y", i
    return MPO(os, sites)
end

function rxx_gate(sites, i::Int, j::Int, ϕ::Float64)
    os = OpSum()
    os += cos(ϕ / 2.0), "I", i, "I", j
    os += -im * sin(ϕ / 2.0), "X", i, "X", j
    return MPO(os, sites)
end

function ryy_gate(sites, i::Int, j::Int, ϕ::Float64)
    os = OpSum()
    os += cos(ϕ / 2.0), "I", i, "I", j
    os += -im * sin(ϕ / 2.0), "Y", i, "Y", j
    return MPO(os, sites)
end

function rzz_gate(sites, i::Int, j::Int, ϕ::Float64)
    os = OpSum()
    os += cos(ϕ / 2.0), "I", i, "I", j
    os += -im * sin(ϕ / 2.0), "Z", i, "Z", j
    return MPO(os, sites)
end

function cx_gate(sites, control::Int, target::Int)
    os = OpSum()
    os += 0.5, "I", control, "I", target
    os += 0.5, "Z", control, "I", target
    os += 0.5, "I", control, "X", target
    os += -0.5, "Z", control, "X", target
    return MPO(os, sites)
end

function proj_0_gate(sites, i::Int)
    os = OpSum()
    os += 0.5, "I", i
    os += 0.5, "Z", i
    return MPO(os, sites)
end

function proj_1_gate(sites, i::Int)
    os = OpSum()
    os += 0.5, "I", i
    os += -0.5, "Z", i
    return MPO(os, sites)
end


proj_1_gate (generic function with 1 method)

In [3]:
function create_sites(n::Int)
    return [Index(2, "Qubit,i$i") for i in 1:n]
end
# check if the MPO creation was sucessful
function inspect_mpo(mpo::MPO, sites::Vector)
    N = length(sites) 

    println("\nManual Inspection of MPO:")
    for site_idx in 1:N
        println("Site $site_idx Operator:")
        println(mpo[site_idx])
    end
end

inspect_mpo (generic function with 1 method)

In [4]:
function parse_subcircuits(json_filename::String)
    json_data = JSON.parsefile(json_filename)
    
    subexp_dict = Dict{String, Vector{Dict{String, Any}}}()
    qubit_num_dict = Dict{String, Int}()
    
    for entry in json_data
        category = entry["Subexperiment"]
        qubit_num = entry["Qubit number"]
        qubit_range = entry["Qubit range"]
        index = entry["Subcircuit"]
        operations_raw = entry["Operations"]
        
        offset = qubit_range[1] - 1

        # Ensure consistency of qubit numbers within the same category
        if haskey(qubit_num_dict, category)
            if qubit_num_dict[category] != qubit_num
                error("Inconsistent qubit numbers in category $category")
            end
        else
            qubit_num_dict[category] = qubit_num
        end
        
        processed_operations = Vector{Dict{String, Any}}()
        for op in operations_raw
            name = op["Name"]
            qubits = op["Qubits"]
            angle_array = op["Angle"]
            
            adjusted_qubits = [q - offset for q in qubits]
            
            angle = if length(angle_array) == 1
                        angle_array[1]
                    else
                        NaN  
                    end
            
            processed_op = Dict{String, Any}(
                "Name" => name,
                "Angle" => angle,
                "Qubits" => adjusted_qubits
            )
            
            push!(processed_operations, processed_op)
        end
        
        subcircuit = Dict{String, Any}(
            "Subcircuit" => index + 1,
            "Operations" => processed_operations
        )
        
        # Add the subcircuit to the subexp_dict
        if haskey(subexp_dict, category)
            push!(subexp_dict[category], subcircuit)
        else
            subexp_dict[category] = [subcircuit]
        end
    end
    
    sorted_categories = sort(collect(keys(subexp_dict)))
    
    organized_data = Dict{String, Dict{String, Any}}()
    
    for category in sorted_categories
        sorted_subcircuits = sort(subexp_dict[category], by = sc -> sc["Subcircuit"])
        organized_data[category] = Dict{String, Any}(
            "Qubit number" => qubit_num_dict[category],
            "Subcircuits" => sorted_subcircuits
        )
    end
    
    return organized_data
end

#Fixed qubit range to always be [1, n_max], and fixed subcircuit index to start at 1



parse_subcircuits (generic function with 1 method)

In [5]:
function build_mpo_sequence(sites, operations, basis_gates)
    """
    Builds a list of Matrix Product Operators (MPOs) based on the provided operations.
    """
    mpo_list = []

    for op_dict in operations
        gate_type = op_dict["Name"]
        qubits = op_dict["Qubits"]
        angle = get(op_dict, "Angle", NaN) 

        if gate_type in basis_gates
            if gate_type == "h"
                i = qubits[1]
                push!(mpo_list, h_gate(sites, i))

            elseif gate_type == "rx"
                θ = angle
                i = qubits[1]
                push!(mpo_list, rx_gate(sites, i, θ))

            elseif gate_type == "ry"
                θ = angle
                i = qubits[1]
                push!(mpo_list, ry_gate(sites, i, θ))

            elseif gate_type == "rz"
                θ = angle
                i = qubits[1]
                push!(mpo_list, rz_gate(sites, i, θ))

            elseif gate_type == "rxx"
                φ = angle
                i, j = qubits[1], qubits[2]
                push!(mpo_list, rxx_gate(sites, i, j, φ))

            elseif gate_type == "rzz"
                φ = angle
                i, j = qubits[1], qubits[2]
                push!(mpo_list, rzz_gate(sites, i, j, φ))

            elseif gate_type == "ryy"
                φ = angle
                i, j = qubits[1], qubits[2]
                push!(mpo_list, ryy_gate(sites, i, j, φ))

            elseif gate_type == "cx"
                control, target = qubits[1], qubits[2]
                push!(mpo_list, cx_gate(sites, control, target))

            end
    
            elseif gate_type == "measure"
                push!(mpo_list, ("measure", qubits[1])) 
        
        
        else
            error("Unsupported gate type: $gate_type. Supported gates are: $(join(basis_gates, ", ")), Measure")
        end
    end

    return mpo_list
end


function apply_measurement(rho, sites, index)
    M0 = proj_0_gate(sites, index)
    M1 = proj_1_gate(sites, index)
    return apply(apply(M0, rho), dag(M0)) - apply(apply(M1, rho), dag(M1))
end


# Combined function to build MPO sequences and apply them to rho
function mpo_sequence_apply(circ_data, basis_gates)
    all_mpo_sequences = Dict{String, Vector{Any}}()
    all_rhos = Dict{String, Vector{MPO}}()
    observables = Dict{String, MPO}()

    for label in keys(circ_data)
        n_qubits = circ_data[label]["Qubit number"]
        sites_label = create_sites(n_qubits)

        initial_psi = productMPS(sites_label, "0")
        initial_rho = outer(initial_psi, initial_psi')

        label_mpo_sequences = Vector{Any}()
        label_rhos = Vector{MPO}()

        os = OpSum()
        for i in 1:n_qubits
            os += "Z", i
        end

        Zmpo = MPO(os, sites_label)

        for subcircuit in circ_data[label]["Subcircuits"]
            operations = subcircuit["Operations"]
            index = subcircuit["Subcircuit"]

            mpo_sequence = build_mpo_sequence(sites_label, operations, basis_gates)
            push!(label_mpo_sequences, mpo_sequence)
            
            # now we apply the MPO sequence
            rho = initial_rho
            for mpo in mpo_sequence
                if typeof(mpo) != MPO && mpo[1] == "measure"
                    qubit_index = mpo[2]
                    rho = apply_measurement(rho, sites_label, qubit_index)
                elseif isa(mpo, MPO)
                    rho = apply(apply(mpo, rho), dag(mpo))
                end
            end 
            println("Completed applying MPO sequence for subcircuit $label$index.")
            push!(label_rhos, rho)
        end

        all_mpo_sequences[label] = label_mpo_sequences
        all_rhos[label] = label_rhos
        observables[label] = Zmpo
    end

    return (all_mpo_sequences, all_rhos, observables)
end



mpo_sequence_apply (generic function with 1 method)

In [ ]:
json_filename = "subcircuits.json"  
basis_gates = ["h", "rx", "ry", "rz", "rxx", "rzz", "ryy", "cx"]

circuits_data = parse_subcircuits(json_filename);
all_mpo_sequences, all_rhos, observables = mpo_sequence_apply(circuits_data,basis_gates);

In [ ]:
coefs1 = [0.9605304970014426, 0.1947091711543252, -0.1947091711543252, 0.1947091711543252, -0.1947091711543252, 0.03946950299855745]
coefs = [0.9900332889206209, 0.09933466539753062, -0.09933466539753062, 0.09933466539753062, -0.09933466539753062, 0.009966711079379185]


exp_val_knitting = sum(coefs[i] * (inner(observables["A"],all_rhos["A"][i]; maxdim = 1000, cutoff = 1e-16) + inner(observables["B"],all_rhos["B"][i];  maxdim = 1000, cutoff = 1e-16)) for i in 1:length(coefs)).re
